In [ ]:
!pip install openai

from openai import OpenAI
from google.colab import userdata

# Instantiate OpenAI client (ensure your OpenAI API key is set in your environment variables or pass directly here).
client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

VIDEOID = 'i-txsBoTJtI'
GOOGLE_APIKEY = userdata.get('YT_APIKEY')

In [ ]:
!pip install youtube-transcript-api

from youtube_transcript_api import YouTubeTranscriptApi

def get_video_transcript(video_id):
  try:
      # Fetching the transcript
      transcript = YouTubeTranscriptApi.get_transcript(video_id,  languages=['hi', 'en'])

      # Combine all text entries into a single string
      full_transcript = " ".join(entry['text'] for entry in transcript)
      return full_transcript
  except Exception as e:
      print(f"Failed to fetch transcript: {e}")
      return ""

def get_transcript_summary(transcript):
  response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Provide a detailed summary of the given youtube video transcript."},
            {"role": "user", "content": transcript}
        ]
    )
  return response.choices[0].message.content

video_transcript = get_video_transcript(VIDEOID)
transcript_summary = get_transcript_summary(video_transcript)
print(transcript_summary)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 9.5 MB/s eta 0:00:00


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from googleapiclient.discovery import build

# Function to get YouTube comments
def get_comments(video_id, api_key):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments = []
    response = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        textFormat="plainText",
        maxResults=100
    ).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        if 'nextPageToken' in response:
            response = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                pageToken=response['nextPageToken'],
                maxResults=100
            ).execute()
        else:
            break

    return comments

video_comments = get_comments(VIDEOID, GOOGLE_APIKEY)
print(video_comments)

In [ ]:
# Function to split comments into manageable batches
def batch_comments(comments, max_tokens=2048):
    batches = []
    current_batch = []
    current_length = 0

    for comment in comments:
        comment_length = len(comment.split())
        if current_length + comment_length > max_tokens:
            batches.append(current_batch)
            current_batch = [comment]
            current_length = comment_length
        else:
            current_batch.append(comment)
            current_length += comment_length

    if current_batch:
        batches.append(current_batch)

    return batches

# Function to get summaries from OpenAI
def get_comments_summaries(batches):
    summaries = []

    for batch in batches:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Summarize the following comments while keeping the detailed context."},
                {"role": "user", "content": " ".join(batch)}
            ]
        )
        print(response)
        summaries.append(response.choices[0].message.content)

    return summaries

# Function to create final summary from summaries
def create_final_summary(summaries, transcript_summary):
    summary_text = " ".join(summaries)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
              {"role": "system", "content": f"This is the summary of a YouTube video's transcript: {transcript_summary}. A user has commented on the video. Your task is to analyze this comment in the context of the video transcript. Based on the comment content and its relation to the transcript, please provide detailed insights, addressing these key points:\n1. Identify positive aspects of the video that the comment highlights and link these to specific parts of the transcript where possible.\n2. Identify any criticisms or areas for improvement mentioned in the comment, and relate these to relevant sections of the transcript.\n3. Based on the feedback or suggestions in the comment, recommend new content ideas or topics for future videos that align with the viewer's interests and the overall content strategy but don't make up things from your side unnecessarily. Ensure your analysis is clear and includes specific examples from both the comment and the transcript to support your insights."},
              {"role": "user", "content": summary_text}
        ]
    )
    return response.choices[0].message.content

In [ ]:
batches = batch_comments(video_comments)
summaries = get_comments_summaries(batches)
final_comments_summary = create_final_summary(summaries, transcript_summary)
print(final_comments_summary)

In [ ]:
def get_sentiments(comments):
    sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}

    for comment in comments:
        # Perform sentiment analysis on each comment individually
        try:
            sentiments = sentiment_analyzer(comment)
            # Assuming the first result is the relevant one if multiple are returned
            sentiment = sentiments[0] if isinstance(sentiments, list) and sentiments else None
        except Exception as e:
            print(f"Error in sentiment analysis: {e}")
            sentiment = None

        # Tally sentiments based on the analysis result
        if sentiment and sentiment['label'] == 'POSITIVE' and sentiment['score'] > 0.9:
            sentiment_counts['positive'] += 1
        elif sentiment and sentiment['label'] == 'NEGATIVE' and sentiment['score'] > 0.9:
            sentiment_counts['negative'] += 1
        else:
            sentiment_counts['neutral'] += 1

    return sentiment_counts

In [ ]:
# Function to plot the sentiment distribution
def plot_sentiment_distribution(sentiment_counts):
    labels = sentiment_counts.keys()
    sizes = sentiment_counts.values()
    colors = ['gold', 'lightcoral', 'lightskyblue']
    explode = (0.1, 0, 0)  # explode 1st slice

    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=140)

    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

In [ ]:
!pip install transformers matplotlib

import matplotlib.pyplot as plt
from transformers import pipeline

# Initialize the sentiment-analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

sentiment_counts = get_sentiments(video_comments)
plot_sentiment_distribution(sentiment_counts)